In [5]:
import numpy as np

In [6]:
# Matriks K44 (8x8 matriks biner)
K44 = np.array([
    [0, 1, 0, 1, 0, 1, 1, 1],
    [1, 0, 1, 0, 1, 0, 1, 1],
    [1, 1, 0, 1, 0, 1, 0, 1],
    [1, 1, 1, 0, 1, 0, 1, 0],
    [0, 1, 1, 1, 0, 1, 0, 1],
    [1, 0, 1, 1, 1, 0, 1, 0],
    [0, 1, 0, 1, 1, 1, 0, 1],
    [1, 0, 1, 0, 1, 1, 1, 0]
])
# Konstanta CAES (8-bit)
CAES = np.array([1, 1, 0, 0, 0, 1, 1, 0])


In [7]:
irreducible_poly = 0x11b  # Polinomial AES x^8 + x^4 + x^3 + x + 1


# Langkah 2: Fungsi Utilitas

In [8]:
def gf_inverse(x, poly=irreducible_poly):
    for i in range(256):
        if (x * i) % 256 == 1:
            return i
    return 0


In [9]:
def affine_transformation(x_inv, K44, CAES):
    x_inv_bits = np.array([int(b) for b in format(x_inv, '08b')])
    result = (np.dot(K44, x_inv_bits) + CAES) % 2
    return int("".join(map(str, result)), 2)


In [10]:
def construct_sbox44(K44, CAES):
    sbox = []
    for i in range(256):
        x_inv = gf_inverse(i)
        sbox.append(affine_transformation(x_inv, K44, CAES))
    return sbox


# Langkah 3: Evaluasi S-Box

In [11]:
def calculate_nonlinearity(sbox):
    max_nl = 0
    for i in range(256):
        hamming_dist = np.sum([bin(i ^ x).count('1') for x in sbox])
        max_nl = max(max_nl, hamming_dist)
    return max_nl


In [12]:
def calculate_sac(sbox):
    sac_score = 0
    for i in range(256):
        flipped_bits = [sbox[i ^ (1 << j)] for j in range(8)]
        sac_score += np.mean([bin(x).count('1') / 8.0 for x in flipped_bits])
    return sac_score / 256


In [13]:
def calculate_lap(sbox):
    lap = 0
    for i in range(256):
        for j in range(256):
            correlation = sum([(bin(x & i).count('1') ^ bin(sbox[x] & j).count('1')) == 0 for x in range(256)])
            lap = max(lap, abs(correlation - 128) / 256)
    return lap


In [14]:
def calculate_dap(sbox):
    dap = 0
    for delta_in in range(256):
        for delta_out in range(256):
            diff = sum([sbox[x] ^ sbox[x ^ delta_in] == delta_out for x in range(256)])
            dap = max(dap, diff / 256)
    return dap


# Langkah 4: Eksekusi

In [15]:
sbox44 = construct_sbox44(K44, CAES)


In [16]:
nl = calculate_nonlinearity(sbox44)
sac = calculate_sac(sbox44)
lap = calculate_lap(sbox44)
dap = calculate_dap(sbox44)

print(f"Nonlinearity (NL): {nl}")
print(f"Strict Avalanche Criterion (SAC): {sac}")
print(f"Linear Approximation Probability (LAP): {lap}")
print(f"Differential Approximation Probability (DAP): {dap}")


KeyboardInterrupt: 